In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
##load IMDB
max=feature=1000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
x_train.shape,y_train.shape

((25000,), (25000,))

In [26]:
##Inspect
sample_x=x_train[2]
sample_y=y_train[2]

In [27]:
##Mapping
word_index=imdb.get_word_index()
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

for i in sample_x:
    print(reverse_word_index.get(i-3,'?'),end=' ')

? this has to be one of the worst films of the ? when my friends i were watching this film being the ? audience it was ? at we just ? watched the first half an hour with our ? ? the ? at how bad it really was the rest of the time everyone else in the ? just started talking to each other ? or ? ? into their ? that they actually ? money they had ? working to watch this ? ? for a film it must have looked like a great idea on ? but on film it looks like no one in the film has a ? what is going on crap acting crap ? i can't get across how ? this is to watch save yourself an hour a bit of your life 

In [34]:
sample_y

0